## Decision Analytics Final Project
### Question: What is the best combination of tourist attractions in NYC that a travel agency can put together in order to provide the best tour experience? 
#### Team Members: Megan, Jhanvi, Hannah, Kaia

In [1]:
import numpy as np
from gurobipy import *
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# prepare model parameters and sets

# parameters
p = [400, 500, 800, 1000] # subject to change when incorporating uncertainty here
pr = np.genfromtxt('Pb1_prob.csv', delimiter=',', encoding='utf-8-sig')  # (1000,)
d = np.genfromtxt('Pb1_D_stochastic.csv', delimiter=',', encoding='utf-8-sig')  # (4, 1000)

# define index and sets
scenario_set = range(len(pr))  # s, range(0, 1000)
class_set = range(len(d[:, 0]))  # c, range(0, 4)

In [ ]:
# initialize model

model = Model()

In [ ]:
# define the decision variables with their constraints

a = model.addVars(class_set, lb=0, vtype=GRB.INTEGER)  # (40, 1) binary variables
x = model.addVars(class_set, scenario_set, lb=0, vtype=GRB.INTEGER)  # (4, 1000) binary variables

In [ ]:
# set up objective function

pb1_objective = sum(pr[s]*sum(p[c]*x[c,s] for c in class_set) for s in scenario_set)
model.setObjective((pb1_objective), GRB.MAXIMIZE)

In [ ]:
# setup remaining constraints.

# The total capacity of the aircraft cannot be gone over the 190 economy seats
model.addConstr(a[0] + 1.2*a[1] + 1.5*a[2] + 2*a[3] <= 190)

# The number of seats sold for each class c must not go over the assigned capacity a for all scenarios s
for c in class_set:
    for s in scenario_set:
        model.addConstr(x[c,s] <= a[c])

# The number of seats sold for each class c must not go over the demand for class c for all scenarios s
for c in class_set:
    for s in scenario_set:
        model.addConstr(x[c,s] <= d[c,s])

In [ ]:
# Solve the problem

model.optimize()